# Sixth Stage

### Investigate into the impact of flash orders on the simulation results

In [1]:
import sys
sys.path.append('..')
from optimization import *
from flash_order import *

In [2]:
target_stock_code = '0050'
side = 'bid'
ts = 20
tm = 10
max_dur = 200
trick_thres = 100

In [3]:
# First get simulation results
sim_res = read_pkl_helper(os.path.join(PROJECT_DIR, f'optres/foms=0/stock_code={target_stock_code}/side={side}/ts={ts}/tm={tm}', 'full_res.pkl'))

In [4]:
# Get flash orders
foc = FlashOrderCalculator(target_stock_code)
foc.classify(max_dur_ms=max_dur, trick_trade_thres_ms=trick_thres)

2022-12-24 04:31:53.239 | INFO     | get_data.py         :   36 | Got cache at /Users/anthonywah/Projects/mafs6100-order-placement-strategy/cache/0050.pkl - 5.81s
ClassifyingFlashOrder: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [00:08<00:00, 21.72it/s]


In [5]:
len(sim_res)

49945

In [6]:
# For each simulation round, check if there is a flash order during the order placement period

for col in ['fo', 'fo_bid', 'fo_ask']:  # Record matched fo in index lists
    if col in sim_res.columns:
        sim_res = sim_res.drop(col, axis=1).reset_index(drop=True)
    sim_res.loc[:, col] = [[] for i in range(len(sim_res))]

one_fod = foc.fod[max_dur][trick_thres]
for fo in tqdm.tqdm(one_fod, desc="FindingFlashOrders", ncols=200, total=len(one_fod)):
    matched = sim_res.loc[(sim_res['start_ts'] <= fo['start_ms']) & (sim_res['fill_ts'] >= fo['end_ms'])].index
    if not len(matched):
        continue
    
    # Store case indexes
    for i in matched:
        sim_res.at[i, 'fo'].append(fo['case_index'])
    
    # Side breakdown
    if fo['side'] == 'bid':
        sim_res.at[i, 'fo_bid'].append(fo['case_index'])
    else: 
        sim_res.at[i, 'fo_ask'].append(fo['case_index'])

FindingFlashOrders: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10434/10434 [00:05<00:00, 2073.29it/s]


In [7]:
# Get simulation rounds with flash orders
sim_res_dict = {
    'Normal'.ljust(15): sim_res.copy(), 
    'With FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) > 0, :].reset_index(drop=True), 
    'No FO'.ljust(15): sim_res.loc[sim_res['fo'].apply(len) == 0, :].reset_index(drop=True), 
    'With bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) > 0, :].reset_index(drop=True), 
    'No bid FO'.ljust(15): sim_res.loc[sim_res['fo_bid'].apply(len) == 0, :].reset_index(drop=True), 
    'With ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) > 0, :].reset_index(drop=True), 
    'No ask FO'.ljust(15): sim_res.loc[sim_res['fo_ask'].apply(len) == 0, :].reset_index(drop=True), 
}

In [8]:
# Calculation % of different fill types
log_info(f'Cases breakdown: ')
for k, i_sim_res in sim_res_dict.items():
    count_dict = i_sim_res.groupby('case')['date'].count().to_dict()
    msg = f'{k}: Count = {len(i_sim_res):>7}'
    for case in ['INIT', 'EOQREP', 'REP', 'TAKE']:
        msg += f' ; {case} = {count_dict.get(case, 0.) * 100 / len(i_sim_res):>6.2f}%'
    msg += f' ; Avg PnL = {i_sim_res["pnl"].mean():>6.3f} ; Stdev = {i_sim_res["pnl"].std():>6.3f}'
    log_info(msg)

2022-12-24 04:32:06.953 | INFO     | 652576031.py        :    2 | Cases breakdown: 
2022-12-24 04:32:06.960 | INFO     | 652576031.py        :    9 | Normal         : Count =   49945 ; INIT =   5.75% ; EOQREP =   4.69% ; REP =  28.66% ; TAKE =  60.90% ; Avg PnL = -2.085 ; Stdev =  3.115
2022-12-24 04:32:06.963 | INFO     | 652576031.py        :    9 | With FO        : Count =    4321 ; INIT =   7.94% ; EOQREP =   4.84% ; REP =  34.16% ; TAKE =  53.07% ; Avg PnL = -2.456 ; Stdev =  4.471
2022-12-24 04:32:06.969 | INFO     | 652576031.py        :    9 | No FO          : Count =   45624 ; INIT =   5.55% ; EOQREP =   4.67% ; REP =  28.13% ; TAKE =  61.65% ; Avg PnL = -2.049 ; Stdev =  2.953
2022-12-24 04:32:06.970 | INFO     | 652576031.py        :    9 | With bid FO    : Count =    1450 ; INIT =   4.41% ; EOQREP =   5.03% ; REP =  20.41% ; TAKE =  70.14% ; Avg PnL = -4.666 ; Stdev =  5.257
2022-12-24 04:32:06.976 | INFO     | 652576031.py        :    9 | No bid FO      : Count =   48495 ;

---

### Compare simulation results with consideration of flash orders

In [9]:
target_stock_code = '0050'
side = 'bid'
ts = 20
tm = 10
foms = 300

In [13]:
files_df

,foms,stock_code,side,ts,tm,path
0,0,2330,ask,11,5,/Users/anthonywah/Projects/mafs6100-order-plac...
1,0,2330,ask,11,5,/Users/anthonywah/Projects/mafs6100-order-plac...
2,0,2330,ask,11,5,/Users/anthonywah/Projects/mafs6100-order-plac...
3,0,2330,ask,11,5,/Users/anthonywah/Projects/mafs6100-order-plac...
4,0,2330,ask,11,5,/Users/anthonywah/Projects/mafs6100-order-plac...
...,...,...,...,...,...,...
229315,100,0050,bid,5,1,/Users/anthonywah/Projects/mafs6100-order-plac...
229316,100,0050,bid,5,1,/Users/anthonywah/Projects/mafs6100-order-plac...
229317,100,0050,bid,5,1,/Users/anthonywah/Projects/mafs6100-order-plac...
229318,100,0050,bid,5,1,/Users/anthonywah/Projects/mafs6100-order-plac...


In [10]:
files_df = get_path_df()

# Limit to certain params only
files_df = files_df.loc[(files_df['ts'].astype(int) <= 20) & (files_df['tm'].astype(int) <= 20)].reset_index(drop=True)

gb = files_df.groupby(['stock_code', 'foms', 'side', 'ts', 'tm'])

In [11]:
res_dict = {}
for k, df in tqdm.tqdm(gb, desc='GettingSimData', ncols=200, total=len(gb.groups)):
    if k not in res_dict.keys():
        res_dict[k] = get_sim_res(path_ls=df['path'].unique().tolist())

GettingSimData: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1260/1260 [00:13<00:00, 90.95it/s]


In [12]:
sim_res_w_fo = read_pkl_helper(os.path.join(PROJECT_DIR, f'optres/foms={foms}/stock_code={target_stock_code}/side={side}/ts={ts}/tm={tm}', 'full_res.pkl'))
sim_res_wo_fo = read_pkl_helper(os.path.join(PROJECT_DIR, f'optres/foms=0/stock_code={target_stock_code}/side={side}/ts={ts}/tm={tm}', 'full_res.pkl'))

FileNotFoundError: [Errno 2] No such file or directory: '/Users/anthonywah/Projects/mafs6100-order-placement-strategy/optres/foms=300/stock_code=0050/side=bid/ts=20/tm=10/full_res.pkl'

In [ ]:
sim_res_w_fo['case'].unique()


In [ ]:
sim_res_w_fo.loc[sim_res_w_fo['case'].isna()]

---